In [16]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, Trials, space_eval, STATUS_OK
from data_process.data_transform_processor import DataTransformProcessor
from data_process.feature_engineer import FeatureEngineer
from models.model_flow import ModelFlow
from models.nn_models.dnn import DNN
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.linear_model import ElasticNet
from models.tree_models.lgbm import LGBM
from models.backtest import BackTest
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_properties = pd.read_csv(
    '/Users/shuyangdu/Desktop/ZillowChallenge/data/raw_data/properties_2017.csv', 
    low_memory=False
)

In [3]:
df_properties = FeatureEngineer.rename(df_properties)

In [7]:
df_properties_2016 = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/properties_2016_engineered_20170924.csv')

/Users/shuyangdu/anaconda/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
for col in df_properties_2016.columns:
    if col not in df_properties_engineered.columns:
        print col

logerror


In [19]:
df_properties_engineered.shape

(2985217, 104)

# Add features for properties

In [5]:
df_properties_engineered = FeatureEngineer.add_features_properties(df_properties)

In [6]:
df_properties_engineered.columns.shape

(99,)

In [ ]:
for col in 

# Add filled missing values

In [14]:
for region in ['zip', 'city', 'neighbor', 'county']:
    df_fill = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/properties_fill_region_{}.csv'.format(region))
    df_properties_engineered['region_{}_filled'.format(region)] = df_fill['region_{}'.format(region)]

In [17]:
df_properties_engineered = FeatureEngineer.fill_missing_value_stats(df_properties_engineered)

In [20]:
for col in df_properties_engineered:
    print col, df_properties_engineered[col].isnull().sum() / float(len(df_properties_engineered))

id_parcel 0.0
aircon 0.72686675709
architectural_style 0.997969661837
area_basement 0.999454980995
num_bathroom 0.000990547755825
num_bedroom 0.00098652794755
framing 0.995735318404
quality 0.34966369279
num_bathroom_calc 0.0392453881912
deck 0.994178312665
area_firstfloor_finished 0.931744325454
area_total_finished_calc 0.0151067744824
area_live_finished 0.0885801601693
area_liveperi_finished 0.99743033756
area_total_finished 0.936291063598
area_live_entry_finished 0.931744325454
area_basic 0.992803873219
fips 0.000982173155251
num_fireplace 0.895108462802
num_bathroom_full 0.0392453881912
num_garage 0.701526555691
area_garage 0.701526555691
flag_tub 0.983230029844
heating 0.373859923751
latitude 0.000982173155251
longitude 0.000982173155251
area_lot 0.0913521529591
num_pool 0.819231901734
area_pool 0.990634516687
flag_hot_tub 0.994303261706
flag_spa 0.988926768138
flag_no_tub_or_spa 0.830533257716
zoning_landuse_county 0.00100461708479
zoning_landuse 0.000982173155251
zoning_property

In [21]:
save_path='/Users/shuyangdu/Desktop/ZillowChallenge/data/properties_2017_engineered_20171007.csv'

In [22]:
df_properties_engineered.to_csv(save_path, index=False)

# Merge with transactions and further feature engineering

In [23]:
df_train = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/raw_data/train_2017.csv')

In [28]:
df_merged_0924 = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/df_merged_20170924.csv')

/Users/shuyangdu/anaconda/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
df_merged = FeatureEngineer.merge(df_properties_engineered, df_train)

In [26]:
df_merged = FeatureEngineer.add_features_transactions(df_merged)

In [40]:
df_merged_0924 = FeatureEngineer.add_features_transactions(df_merged_0924)

In [41]:
df_merged_0924.shape

(90275, 109)

In [42]:
df_merged.shape

(77613, 109)

In [47]:
df_merged_concat = pd.concat([df_merged_0924, df_merged], axis=0, ignore_index=True)

In [50]:
df_merged_concat.to_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/df_merged_20171007.csv', index=False)

# ----Computation Heavy----

# Model based fill missing values

In [4]:
y_col = 'region_zip'
X_numerical_cols = ['latitude', 'longitude']
X_categorical_cols = ['region_neighbor', 'region_county', 'region_city']

In [5]:
data_transform_processor = DataTransformProcessor(
    numerical_cols=X_numerical_cols, 
    categorical_cols=X_categorical_cols,
    log_cols=[]
)

In [6]:
idx_train = df_properties[y_col].notnull().values

In [7]:
X_all = data_transform_processor.pre_process(df_properties)

In [8]:
y_all = df_properties[y_col].values

In [9]:
params = {
    'max_bin': 80,
    'learning_rate': 0.0216,
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': len(np.unique(y_all[idx_train])),
    'feature_fraction': 0.94,
    'bagging_fraction': 0.85,
    'bagging_freq': 80,
    'num_leaves': 60,
    'n_estimators': 200,
}

In [10]:
model = LGBM(is_classifier=True, categorical_feature=data_transform_processor.categorical_col_idx, **params)

In [11]:
X_train = data_transform_processor.fit_transform(X_all[idx_train])
y_train = y_all[idx_train]

In [14]:
%%time
model.fit(X_train, y_train)

CPU times: user 1d 7h 50min 1s, sys: 8h 50min 9s, total: 1d 16h 40min 11s
Wall time: 8h 36min 5s


In [15]:
X_pred = data_transform_processor.transform(X_all[~idx_train])

In [16]:
%%time
y_pred = model.predict(X_pred)

CPU times: user 13min 48s, sys: 9.69 s, total: 13min 58s
Wall time: 2min 3s


In [17]:
y_all[~idx_train] = y_pred

In [18]:
df_region = pd.DataFrame(y_all, columns=[y_col])

In [19]:
df_region.to_csv(
    '/Users/shuyangdu/Desktop/ZillowChallenge/data/properties_fill_{}.csv'.format(y_col), 
    index=False
)